In [3]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="sk-e3d9c4c0f1054400a0bb88d4a205bc8a",
    base_url="https://api.deepseek.com",
)


system_prompt = """
Please determine if this message affects any state attributes other than the list of state attributes above. If so, return all attribute names.
Otherwise, return an empty list. The answer should be organized in JSON format. The following is an example of the result of a ZCL message AddGroup. 

EXAMPLE JSON OUTPUT:
{
   'Attributes': ['group_count']
}
"""

user_prompt = """The following is the list of device state attributes.
["zcl_version", ""]

The following is the description of the Zigbee ZCL message MoveHue.
{"description": "This function sends a Move to Hue command to instruct a device to move its ‘current
hue’ attribute to a target hue value in a continuous manner within a given time. The
hue value, direction and transition time are specified in the payload of the command
(see Section 20.6.2).
Since the possible hues are represented on a closed boundary, the target hue can
be reached by moving the attribute value in either direction, up or down (the attribute
value wraps around). Options are also provided for ‘shortest route’ and ‘longest
route’ around the boundary.
The device receiving this message will generate a callback event on the endpoint on
which the Colour Control cluster was registered. The device must first ensure that
‘hue and saturation’ mode is selected by setting the ‘colour mode’ attribute to 0x00,
if required. It can then move the ‘current hue’ value as requested.
You are required to provide a pointer to a location to receive a Transaction Sequence
Number (TSN) for the request. The TSN in the response will be set to match the TSN
in the request, allowing an incoming response to be paired with a request. This is
useful when sending more than one request to the same destination endpoint.
This function can only be used when the ‘current hue’ attribute is enabled in the
Colour Control cluster."
}

"""

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

print(json.loads(response.choices[0].message.content))

{'question': 'What is the purpose of the Zigbee ZCL message MoveHue?', 'answer': "The MoveHue command instructs a device to move its 'current hue' attribute to a target hue value in a continuous manner within a given time, specifying the hue value, direction, and transition time in the payload. It allows for options like 'shortest route' and 'longest route' around the hue boundary and requires the 'colour mode' attribute to be set to 0x00 ('hue and saturation' mode) if necessary. The function also requires a pointer for a Transaction Sequence Number (TSN) to pair requests with responses, especially useful for multiple requests to the same endpoint. This function is only usable when the 'current hue' attribute is enabled in the Colour Control cluster."}


In [1]:
import os
import re
import json
from openai import OpenAI

def get_all_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            file_paths.append(full_path)
    return file_paths


def remove_triple_quoted_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    cleaned_content = re.sub(r'"""(.*?)"""', '', content, flags=re.DOTALL)

    return cleaned_content

def write_list_to_file(filepath, data_list):
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data_list:
            f.write(f"{item}\n")

def read_attributes_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]


def get_filename_without_extension(file_path):
    base_name = os.path.basename(file_path)           # 获取文件名（含后缀）
    name_only = os.path.splitext(base_name)[0]        # 去除后缀
    return name_only

In [5]:
def LLM_prompt(file_content: str):
    client = OpenAI(
        api_key="sk-aff5fd7dc30e419e9b0c1814159fa58a",
        base_url="https://api.deepseek.com",
    )
    
    system_prompt = """
    Please extract all commands defined in the class "ClientCommandDefs(BaseCommandDefs)". The following is a example of a "ClientCommandDefs(BaseCommandDefs)" class.
    
    EXAMPLE INPUT: 
     class ClientCommandDefs(BaseCommandDefs):
        add_response: Final = ZCLCommandDef(
            id=0x00,
            schema={"status": foundation.Status, "group_id": t.Group},
            direction=Direction.Server_to_Client,
        )
        view_response: Final = ZCLCommandDef(
            id=0x01,
            schema={
                "status": foundation.Status,
                "group_id": t.Group,
                "group_name": t.LimitedCharString(16),
            },
            direction=Direction.Server_to_Client,
        )
    
    EXAMPLE JSON OUTPUT:
    {
       "Commands": ["add_response", "view_response"]
    }
    """
    
    user_prompt = f"""The following is a python code that define some Zigbee ZCL clusters with attributes and commands defined in each cluster.
    {file_content}
    """
    
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
            'type': 'json_object'
        }
    )
    
    result = json.loads(response.choices[0].message.content)
    print(result)
    return result

In [6]:
file_paths = get_all_file_paths(os.path.join(os.getcwd(), "zcl/cluster"))
all_result = []

for file_path in file_paths:
    cluster_name = get_filename_without_extension(file_path)
    cleaned_content = remove_triple_quoted_content(file_path)
    result = LLM_prompt(cleaned_content)
    if 'Commands' in result.keys():
        if type(result['Commands']) == list:
            all_result.extend(result['Commands'])
        
        with open(os.path.join(os.getcwd(), f"zcl/result/command/{cluster_name}-clientcmd.json"), "w") as f:
            json.dump(result, f, indent=4)
            
write_list_to_file(os.path.join(os.getcwd(), "zcl/result/command/clientcmd.txt"), all_result)

{'Commands': ['alarm', 'get_alarm_response']}
{'Commands': []}
{'Commands': ['transfer_apdu', 'connect_req', 'disconnect_req', 'connect_status_noti']}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': []}
{'Commands': ['get_alerts_response', 'alerts_notification', 'event_notification']}
{'Commands': []}


KeyboardInterrupt: 